# Step1 安装依赖

In [ ]:
!pip uninstall -y parl  # 说明：AIStudio预装的parl版本太老，容易跟其他库产生兼容性冲突，建议先卸载
!pip uninstall -y pandas scikit-learn # 提示：在AIStudio中卸载这两个库再import parl可避免warning提示，不卸载也不影响parl的使用

!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple gym
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple atari-py # 玩Gym的Atari游戏必装依赖，本次作业使用了Atari的Pong(乒乓球)环境
!pip install paddlepaddle-gpu==1.6.3.post107
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple parl==1.3.1

# 说明：安装日志中出现两条红色的关于 paddlehub 和 visualdl 的 ERROR 与parl无关，可以忽略，不影响使用

Uninstalling pandas-1.0.4:
  Successfully uninstalled pandas-1.0.4
Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 323.3MB 50kB/s 
     |████████████████████████████████| 122kB 36.6MB/s 
  Created wheel for rarfile: filename=rarfile-3.1-cp36-none-any.whl size=24908 sha256=ea5679bb4c0ce09d63b93705981289eaa2d68cf430006406529395a3fe89badc
  Stored in directory: /root/.cache/pip/wheels/23/3c/c8/0215b6a5079492eff3be3f545ae0b0c4a66734c35c9e444eac
Successfully built rarfile
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 522kB 192kB/s 
     |████████████████████████████████| 4.3MB 313kB/s 
     |████████████████████████████████| 450kB 33.0MB/s 
     |████████████████████████████████| 1.1MB 31.6MB/s 
     |████████████████████████████████

In [ ]:
# 检查依赖包版本是否正确
!pip list | grep paddlepaddle
!pip list | grep parl


paddlepaddle-gpu         1.6.3.post107  
parl                     1.3.1          


# Step2 导入依赖

In [ ]:
import os
import gym
import numpy as np

import paddle.fluid as fluid
import parl
from parl import layers
from parl.utils import logger

# Step3 设置超参数

In [ ]:
######################################################################
######################################################################
#
# 1. 请设定 learning rate，尝试增减查看效果
#
######################################################################
######################################################################

LEARNING_RATE = 0.0015

# Step4 搭建Model、Algorithm、Agent架构
* `Agent`把产生的数据传给`algorithm`，`algorithm`根据`model`的模型结构计算出`Loss`，使用`SGD`或者其他优化器不断的优化，`PARL`这种架构可以很方便的应用在各类深度强化学习问题中。

#### （1）Model
`Model`用来定义前向(`Forward`)网络，用户可以自由的定制自己的网络结构。

In [ ]:
class Model(parl.Model):
    def __init__(self, act_dim):
        ######################################################################
        ######################################################################
        self.fc1 = layers.fc(size=800, act='relu')
        self.fc2 = layers.fc(size=act_dim, act='softmax')
        ######################################################################
        ######################################################################

    def forward(self, obs):  # 可直接用 model = Model(5); model(obs)调用
        ######################################################################
        ######################################################################
        h1 = self.fc1(obs)
        out = self.fc2(h1)
        ######################################################################
        ######################################################################
        return out


#### （2）Algorithm
* `Algorithm` 定义了具体的算法来更新前向网络(`Model`)，也就是通过定义损失函数来更新`Model`，和算法相关的计算都放在`algorithm`中。

In [ ]:
from parl.algorithms import PolicyGradient # 直接从parl库中导入PolicyGradient算法，无需重复写算法

#### （3）Agent
* `Agent`负责算法与环境的交互，在交互过程中把生成的数据提供给`Algorithm`来更新模型(`Model`)，数据的预处理流程也一般定义在这里。

In [ ]:
class Agent(parl.Agent):
    def __init__(self, algorithm, obs_dim, act_dim):
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        super(Agent, self).__init__(algorithm)

    def build_program(self):
        self.pred_program = fluid.Program()
        self.learn_program = fluid.Program()

        with fluid.program_guard(self.pred_program):  # 搭建计算图用于 预测动作，定义输入输出变量
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            self.act_prob = self.alg.predict(obs)

        with fluid.program_guard(
                self.learn_program):  # 搭建计算图用于 更新policy网络，定义输入输出变量
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            act = layers.data(name='act', shape=[1], dtype='int64')
            reward = layers.data(name='reward', shape=[], dtype='float32')
            self.cost = self.alg.learn(obs, act, reward)

    def sample(self, obs):
        obs = np.expand_dims(obs, axis=0)  # 增加一维维度
        act_prob = self.fluid_executor.run(
            self.pred_program,
            feed={'obs': obs.astype('float32')},
            fetch_list=[self.act_prob])[0]
        act_prob = np.squeeze(act_prob, axis=0)  # 减少一维维度
        act = np.random.choice(range(self.act_dim), p=act_prob)  # 根据动作概率选取动作
        return act

    def predict(self, obs):
        obs = np.expand_dims(obs, axis=0)
        act_prob = self.fluid_executor.run(
            self.pred_program,
            feed={'obs': obs.astype('float32')},
            fetch_list=[self.act_prob])[0]
        act_prob = np.squeeze(act_prob, axis=0)
        act = np.argmax(act_prob)  # 根据动作概率选择概率最高的动作
        return act

    def learn(self, obs, act, reward):
        act = np.expand_dims(act, axis=-1)
        feed = {
            'obs': obs.astype('float32'),
            'act': act.astype('int64'),
            'reward': reward.astype('float32')
        }
        cost = self.fluid_executor.run(
            self.learn_program, feed=feed, fetch_list=[self.cost])[0]
        return cost


### Step 5 Training && Test（训练&&测试）

In [ ]:
def run_episode(env, agent):
    obs_list, action_list, reward_list = [], [], []
    obs = env.reset()
    while True:
        obs = preprocess(obs) # from shape (210, 160, 3) to (100800,)
        obs_list.append(obs)
        action = agent.sample(obs) # 采样动作
        action_list.append(action)

        obs, reward, done, info = env.step(action)
        reward_list.append(reward)

        if done:
            break
    return obs_list, action_list, reward_list


# 评估 agent, 跑 5 个episode，求平均
def evaluate(env, agent, render=False):
    eval_reward = []
    for i in range(5):
        obs = env.reset()
        episode_reward = 0
        while True:
            obs = preprocess(obs) # from shape (210, 160, 3) to (100800,)
            action = agent.predict(obs) # 选取最优动作
            obs, reward, isOver, _ = env.step(action)
            episode_reward += reward
            if render:
                env.render()
            if isOver:
                break
        eval_reward.append(episode_reward)
    return np.mean(eval_reward)

### Step6 创建环境和Agent，启动训练，保存模型

In [ ]:

# Pong 图片预处理
def preprocess(image):
    """ 预处理 210x160x3 uint8 frame into 6400 (80x80) 1维 float vector """
    image = image[35:195] # 裁剪
    image = image[::2,::2,0] # 下采样，缩放2倍
    image[image == 144] = 0 # 擦除背景 (background type 1)
    image[image == 109] = 0 # 擦除背景 (background type 2)
    image[image != 0] = 1 # 转为灰度图，除了黑色外其他都是白色
    return image.astype(np.float).ravel()


# 根据一个episode的每个step的reward列表，计算每一个Step的Gt
def calc_reward_to_go(reward_list, gamma=0.99):
    """calculate discounted reward"""
    reward_arr = np.array(reward_list)
    for i in range(len(reward_arr) - 2, -1, -1):
        # G_t = r_t + γ·r_t+1 + ... = r_t + γ·G_t+1
        reward_arr[i] += gamma * reward_arr[i + 1]
    # normalize episode rewards
    reward_arr -= np.mean(reward_arr)
    reward_arr /= np.std(reward_arr)
    return reward_arr


# 创建环境
env = gym.make('Pong-v0')
obs_dim = 80 * 80
act_dim = env.action_space.n
logger.info('obs_dim {}, act_dim {}'.format(obs_dim, act_dim))

# 根据parl框架构建agent
######################################################################
######################################################################
#
# 4. 请参考课堂Demo构建 agent，嵌套Model, PolicyGradient, Agent
#
######################################################################
######################################################################
model = Model(act_dim=act_dim)
alg = PolicyGradient(model, lr=LEARNING_RATE)
agent = Agent(alg, obs_dim=obs_dim, act_dim=act_dim)


# 加载模型
# if os.path.exists('./model.ckpt'):
#     agent.restore('./model.ckpt')

for i in range(3000):
    if i == 1000:
      agent.alg.lr = 0.001
    elif i == 2000:
      agent.alg.lr = 0.0005 
           
    obs_list, action_list, reward_list = run_episode(env, agent)
    if i % 10 == 0:
        logger.info("Train Episode {}, Reward Sum {}.".format(i, 
                                            sum(reward_list)))

    batch_obs = np.array(obs_list)
    batch_action = np.array(action_list)
    batch_reward = calc_reward_to_go(reward_list)

    agent.learn(batch_obs, batch_action, batch_reward)
    if (i + 1) % 100 == 0:
        total_reward = evaluate(env, agent, render=False)
        logger.info('Episode {}, Test reward: {}'.format(i + 1, 
                                            total_reward))

# save the parameters to ./model.ckpt
agent.save('./model.ckpt')
    


[06-20 15:39:43 MainThread @<ipython-input-10-a4bcb3f225b8>:30] obs_dim 6400, act_dim 6
[06-20 15:39:43 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-20 15:39:43 MainThread @machine_info.py:86] nvidia-smi -L found gpu count: 1
[06-20 15:39:46 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 0, Reward Sum -20.0.
[06-20 15:40:22 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 10, Reward Sum -21.0.
[06-20 15:40:56 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 20, Reward Sum -21.0.
[06-20 15:41:33 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 30, Reward Sum -20.0.
[06-20 15:42:12 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 40, Reward Sum -20.0.
[06-20 15:42:47 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 50, Reward Sum -20.0.
[06-20 15:43:27 MainThread @<ipython-input-10-a4bcb3f225b8>:58] Train Episode 60, Reward Sum -20.0.
[06-20 15:44:16 MainThread @<ipython-input-10-a4bcb